In [14]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.feature_selection import (
    chi2,
    f_classif,
    r_regression,
    f_regression,
    mutual_info_classif,
    mutual_info_regression,
    SelectKBest,
    SelectPercentile,
    RFE,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create dataset

In [2]:
n_features = 20
n_informative = 10
n_repeated = 2
n_redundant = n_features - n_informative - n_repeated

x, y = make_classification(
    n_samples=100,
    n_features=n_features,
    n_informative=n_informative,
    n_redundant=n_redundant,
    n_repeated=n_repeated,
    shuffle=False,
)

cols = (
    [f"informative_{i+1}" for i in range(n_informative)]
    + [f"redundant_{i+1}" for i in range(n_redundant)]
    + [f"repeated_{i+1}" for i in range(n_repeated)]
)
df = pd.DataFrame(x, columns=cols)
np.random.shuffle(cols)
df = df.loc[:, cols]

df.head()

,informative_3,redundant_1,redundant_6,repeated_2,redundant_2,redundant_4,informative_6,informative_4,redundant_8,informative_9,informative_7,informative_1,informative_5,redundant_5,informative_10,repeated_1,redundant_7,redundant_3,informative_2,informative_8
0,2.184556,5.836486,-5.694648,1.055499,5.368003,-0.225743,1.055499,-1.824451,7.133486,2.013175,-0.213076,2.217145,-0.730753,-4.406577,-3.058126,7.133486,-1.382256,-2.887262,4.961870,2.232432
1,-1.316643,-1.607137,-3.469298,0.440428,2.240010,1.312680,0.440428,0.292198,4.102107,2.053920,-1.044936,-0.925077,1.584566,3.278208,-1.567449,4.102107,-0.861812,-0.389031,0.600676,0.159714
2,-1.794947,3.020817,-2.783226,3.103997,3.596217,-3.089403,3.103997,0.625332,3.487440,-0.797034,-2.897013,0.689951,2.428056,2.905673,-0.089383,3.487440,-1.158112,-1.255294,3.495078,1.574814
3,-3.029316,-1.747230,-1.173629,2.632483,-1.097512,-1.809024,2.632483,1.398997,2.309135,-0.459959,-2.157722,-1.093022,2.066988,7.718312,-1.340471,2.309135,0.978617,-0.804663,-0.075781,-0.076565
4,3.007401,0.100642,-6.822637,1.326029,-2.345675,0.967230,1.326029,-4.448662,7.809589,3.186244,2.644307,1.041832,-2.793339,-2.012615,-4.443933,7.809589,1.441131,-6.828177,1.124956,1.574575


In [19]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(df)
df = pd.DataFrame(x_scaled, columns=cols)
df.head()

,informative_3,redundant_1,redundant_6,repeated_2,redundant_2,redundant_4,informative_6,informative_4,redundant_8,informative_9,informative_7,informative_1,informative_5,redundant_5,informative_10,repeated_1,redundant_7,redundant_3,informative_2,informative_8
0,0.883692,2.094583,-1.261774,-0.062159,1.796188,0.067701,-0.062159,-0.824608,1.559179,0.879267,0.106003,1.943600,-0.771073,-1.407722,-1.542605,1.559179,-0.289995,-0.320333,2.230005,1.118985
1,-0.698372,-0.221825,-0.476071,-0.482217,0.806395,0.502991,-0.482217,0.270826,0.707539,0.898915,-0.303151,-0.304932,0.472810,0.387232,-0.727976,0.707539,-0.080810,0.344131,0.144440,-0.233590
2,-0.914500,1.218365,-0.233840,1.336846,1.235540,-0.742559,1.336846,0.443233,0.534853,-0.475918,-1.214105,0.850760,0.925967,0.300218,0.079760,0.534853,-0.199903,0.113728,1.528571,0.689848
3,-1.472266,-0.265421,0.334460,1.014829,-0.249699,-0.380282,1.014829,0.843630,0.203818,-0.313368,-0.850481,-0.425111,0.731987,1.424318,-0.603937,0.203818,0.658923,0.233584,-0.179048,-0.387777
4,1.255505,0.309625,-1.660033,0.122597,-0.644656,0.405247,0.122597,-2.182722,1.749125,1.444964,1.511421,1.102562,-1.879177,-0.848558,-2.299925,1.749125,0.844823,-1.368514,0.395156,0.689693


In [20]:
df.describe()

,informative_3,redundant_1,redundant_6,repeated_2,redundant_2,redundant_4,informative_6,informative_4,redundant_8,informative_9,informative_7,informative_1,informative_5,redundant_5,informative_10,repeated_1,redundant_7,redundant_3,informative_2,informative_8
count,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
mean,5.329071e-17,-7.438494e-17,1.554312e-16,1.487699e-16,-8.604228e-18,1.998401e-17,1.487699e-16,-8.243406e-17,-4.662937e-17,1.110223e-18,-5.245804e-17,1.298961e-16,-2.553513e-16,-1.976197e-16,1.204592e-16,-4.662937e-17,4.218847e-17,-9.103829e-17,5.995204e-17,3.996803e-17
std,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00,1.005038e+00
min,-2.421711e+00,-2.497186e+00,-2.492033e+00,-2.432241e+00,-2.516299e+00,-2.531558e+00,-2.432241e+00,-2.278090e+00,-2.884240e+00,-2.193390e+00,-2.977465e+00,-2.174941e+00,-2.311191e+00,-2.099967e+00,-2.450175e+00,-2.884240e+00,-2.501674e+00,-2.625991e+00,-2.971284e+00,-2.853512e+00
25%,-5.422872e-01,-7.940848e-01,-6.966137e-01,-6.243523e-01,-6.445424e-01,-6.776141e-01,-6.243523e-01,-6.612465e-01,-7.499369e-01,-7.346884e-01,-7.547424e-01,-7.659211e-01,-7.007722e-01,-7.350775e-01,-6.418795e-01,-7.499369e-01,-7.073832e-01,-6.360690e-01,-6.054454e-01,-6.433846e-01
50%,1.953073e-02,3.539551e-02,-1.217024e-01,3.088508e-02,-1.377956e-01,6.744565e-02,3.088508e-02,6.604207e-02,6.825175e-02,-1.605225e-01,4.134477e-02,-9.065974e-02,9.645219e-02,-8.013583e-02,-1.111352e-01,6.825175e-02,-3.728509e-02,4.673755e-02,-5.315669e-02,3.410508e-02
75%,6.276025e-01,5.839506e-01,6.905140e-01,5.697485e-01,5.158355e-01,6.858505e-01,5.697485e-01,6.530306e-01,7.673878e-01,7.590427e-01,7.517996e-01,6.440707e-01,6.689785e-01,7.207900e-01,5.888214e-01,7.673878e-01,7.502902e-01,5.633161e-01,6.741944e-01,7.903057e-01
max,3.024371e+00,2.538559e+00,2.688468e+00,2.675185e+00,2.753627e+00,2.280617e+00,2.675185e+00,2.371951e+00,1.919981e+00,2.090653e+00,2.538136e+00,3.143340e+00,3.079962e+00,2.198060e+00,2.598593e+00,1.919981e+00,2.249432e+00,2.656604e+00,2.309250e+00,1.962273e+00


# Univariate feature selection

In [21]:
class UnivariateFeatureSelection:

    def __init__(self, n_features, problem_type, scoring):
        """
        Custom univariate feature selection wrapper on different
        univariate feature selection models from sklearn.
        :param n_features: SelectPercentile if float, SelectKBest if int
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """

        # for a given problem type, there are only a few valid scoring methods
        if problem_type == "classification":
            valid_scoring = {
                "chi2": chi2,
                "f_classif": f_classif,
                "mutual_info_classif": mutual_info_classif,
            }
        elif problem_type == "regression":
            valid_scoring = {
                "r_regression": r_regression,
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression,
            }
        else:
            raise Exception(
                "Invalid problem type. Select regression or classification."
            )

        # raise exception if scoring is not valid
        # seach in valid_scoring keys
        if scoring not in valid_scoring.keys():
            raise Exception("Invalid scoring function.")

        # if n_features == int, SelectKBest
        if isinstance(n_features, int):
            self.selection = SelectKBest(valid_scoring[scoring], k=n_features)
        # if n_features == float, SelectPercentile
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring], percentile=round(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature.")

        self.feature_names = None

    def fit(self, X, y):
        
        # Fit the feature selector
        self.selection.fit(X, y)
        
        # Check if X is a DataFrame for .columns to work
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns[self.selection.get_support()]
        else:
            message = "X must be a pandas DF to extract feature names."
            raise Exception(message)
        
        # Save feature names
        self.feature_names = X.columns[self.selection.get_support()]

        return self

    def transform(self, X):
        return self.selection.transform(X)

    def fit_transform(self, X, y):
        # we have to fit first to be able to obtain feature names
        self.fit(X, y)
        return self.transform(X)

    def get_feature_names(self):
        # if fit is not yet called, raise exception
        if self.feature_names is None:
            raise Exception("Must call fit method first.")
        return self.feature_names

In [22]:
scorings = ["f_classif", "mutual_info_classif"]
for scoring in scorings:
    f_selection = UnivariateFeatureSelection(n_features=10, problem_type="classification", scoring=scoring)
    f_selection.fit_transform(df, y)
    print(f"scoring {scoring}")
    print(f"selected features = {sorted(f_selection.get_feature_names())}")
    print("=" * 100)

scoring f_classif
selected features = ['informative_1', 'informative_10', 'informative_2', 'informative_3', 'informative_4', 'informative_8', 'redundant_1', 'redundant_2', 'redundant_4', 'redundant_5']
scoring mutual_info_classif
selected features = ['informative_10', 'informative_2', 'informative_5', 'informative_7', 'informative_8', 'redundant_1', 'redundant_2', 'redundant_5', 'redundant_8', 'repeated_1']


# Recursive feature elimination